# HW3 GEE_Whiz: Dataset Overview and Use Case Examples
## EDS 220, Fall 2021

## Mangrove Canopy Coverage from LANDSAT 8 Before and After Restoration

## Authors
- Allie Cole, UC Santa Barbara (icole@bren.ucsb.edu) <br>
  https://alliecole.github.io/
- Clarissa Boyajia, UC Santa Barbara (cboyajian@bren.ucsb.edu) <br>
  https://cboyajian.github.io/
- Scout Leonard, UC Santa Barbara (scout@bren.ucsb.edu) <br>
  https://scoutcleonard.github.io/

## Table of Contents

[1. Purpose](#purpose)

[2. Dataset Description](#overview)

[3. Data I/O](#io)

[4. Metadata Display and Basic Visualization](#display)

[5. Use Case Examples](#usecases)

[6. Create Binder Environment](#binder)

[7. References](#references)

<a id='purpose'></a> 
### Notebook Purpose

This notebook was created to give an overview and short tutorial on how to calculate mangrove canopy coverage percentages using data from the NASA & USGS Landsat 8 dataset. For the tutorial, we will focus on the Cacheu River Mangroves Natural Park in the Republic of Guinea-Bissau in West Africa where restoration efforts began in 2017.

<a id='overview'></a> 
### Dataset Description

The Landsat 8 data collection is supported jointly by NASA and hosted by the USGS. It is accessible through the Google Earth Engine (GEE) API. Landsat 8 data can be downloaded in many different formats depending on how you want to manipulate it. The data includes 9 bands: 5 visible and near-infared (NIR) bands, 2 short-wave infared (SWIR), and 2 thermal infared (TIR) bands. 

Various products offer varying combinations of bands. The data found in these products comes from USGS in the form of .TIFFs and .JPEGs.

This dataset includes fully global coverage every 16 days. It incldues 

This portion of the notebook should contain a summary description of your chosen environmental dataset. In a few paragraphs, discuss:
- The creators of the dataset: NASA/NOAA/other government agency? Nonprofit? etc.
- Major characteristics of the dataset: global coverage? Spatial resolution? Temporal resolution? Creation date? 
- The file format(s) used to store the data: netCDF? CSV? Other?
- The source/archive you will be using to retrieve the data: Google Earth Engine? Agency data portal? Other API?
- Any known issues with data quality that might be expected to impact the results

Include links to any external resources needed to access the data here, including either the location of files stored on an external server you've set up or the access URL for a pre-existing repository. You can also include any example images you find useful for motivating the choice of dataset (optional).


<a id='io'></a> 
### Dataset Input/Output 

Next, provide code to read in the data necessary for your analysis. This should be in the following order:

1) Import all necessary packages (matplotlib, numpy, etc)

2) Set any parameters that will be needed during subsequent portions of the notebook. Typical examples of parameters include:
- names of any directories where data are stored
- ranges of years over which data are valid
- any thresholds or latitude/longitude ranges to be used later (e.g. dimensions of NINO3.4 region, threshold SSTA values for El Nino, etc.)

3) Read in the data! If the data files are very large, you may want to consider subsetting the portion of files to be read in (see examples of this during notebooks provided in Weeks 7 and 8).

_Since we will be running these notebooks in class during Weeks 9 and 10_, here is a good rule of thumb: It's good to aim for a relatively short amount of time needed to read in the data, since otherwise we'll be sitting around waiting for things to load for a long time. A  minute or two for data I/O is probably the max you'll want to target!

In [10]:
#import all necessary packages 
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [11]:
#authorize google earth engine access 
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWi6Wv3mtiLIyQVXyiw1oi7QXqEWQtGFLWIuc4yP2g4jpZIS30ZTWoU



Successfully saved authorization token.


In [12]:
gdat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

<a id='display'></a> 
### Metadata Display and Basic Visualization

Next, provide some example commands to take a quick look at what is in your dataset. We've done some things along these lines in class by now, but you should include at least one of:

- Metadata display: commands to indicate a) which variables are included in the dataset and their names; b) coordinate information associated with the data variables; c) other important metadata parameters (site names, etc); and d) any important information on missing data
- Basic visualization: a "quick and dirty" plot showing generally what the data look like. Depending on your dataset, this could be either a time series or a map (no fancy coordinate reference system/projection needed yet).

#### Metadata Display

In [7]:
# pull the first image in the collection 
testimg = gdat.first()

The code chunk below uses the first image in the collection to call the variables included in the images of our Landsat 8 data:

In [8]:
#extract a list containing the names of the bands 
bands = testimg.bandNames()
str(bands.getInfo())

"['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']"

We can see from the returned results that the dataset has 12 variables. Their names are

- `B1` : coastal aerosol
- `B2` : blue
- `B3`: green
- `B4`: red
- `B5` : near infared
- `B6` : shortwave infared 1
- `B7` : shortwave infared 2
- `B8` : band 8 panochromatic
- `B9`: cirrus
- `B10` : thermal infared 1
- `B11` : thermal infared 2
- `sr_aerosol`: aerosol attricutes
- `pixel_qa` : pixel quality attributes generated from the CFMASK algorithm 
- `radsat_qa`: radiometric saturation QA

In [9]:
#select mangrove region (Sundarabans, in Bangladesh) to explore Landsat8 data for

#lat and lon for Sundarabans mangrove forest
sundarbans_lon = 89
sundarbans_lat = 21.37

#create point of interest for Sundarabans 
sundarbans_poi = ee.Geometry.Point(sundarbans_lon, sundarbans_lat)
scale = 1000   # scale in m

In [10]:
#explore a data feature of the Landsat images; we chose Band 2
B2 = gdat.select('B2')
sundarbans_B2 = B2.getRegion(sundarbans_poi, scale).getInfo()

In [11]:
#create a dataframe using Band 2 measurements for Sundarabans mangrove forest over time 
df = pd.DataFrame(sundarbans_B2)

print(df)

                        0          1          2              3     4
0                      id  longitude   latitude           time    B2
1    LC08_138045_20130708  89.000587  21.366429  1373258003390  1091
2    LC08_138045_20130724  89.000587  21.366429  1374640402070  1587
3    LC08_138045_20130825  89.000587  21.366429  1377405206730  1102
4    LC08_138045_20130910  89.000587  21.366429  1378787604620   198
..                    ...        ...        ...            ...   ...
156  LC08_138045_20210714  89.000587  21.366429  1626237078562  3196
157  LC08_138045_20210815  89.000587  21.366429  1629001892732  3953
158  LC08_138045_20210831  89.000587  21.366429  1630384297322  2219
159  LC08_138045_20210916  89.000587  21.366429  1631766701206   951
160  LC08_138045_20211002  89.000587  21.366429  1633149105920   550

[161 rows x 5 columns]


Basic Visualization: 

In [4]:
test_pt = ee.Geometry.Point([-16.283462202, 12.1582165861])

In [5]:
test_filt = gdat.filterBounds(test_pt)

In [6]:
visParams = {'bands': ['B4', 'B3', 'B2'],
             'min': 0,
             'max': 0.5
            }

In [7]:
test_map = geemap.Map(center=[12.1582165861, -16.283462202], zoom=12)
test_map

Map(center=[12.1582165861, -16.283462202], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [8]:
test_map.addLayer(test_filt, visParams)

<a id='usecases'></a> 
### Use Case Examples

This is the "meat" of the notebook, and what will take the majority of the time to present in class. This section should provide:
1) A plain-text summary (1-2 paragraphs) of the use case example you have chosen: include the target users and audience, and potential applicability. For example, the Week 7 SST exercise might discuss how the state of the ENSO system can be important for seasonal weather forecasts/coral bleaching outlooks, then mention the typical diagnostics associated with ENSO (i.e. identification of El Nino/La Nina events).

2) Markdown and code blocks demonstrating how one walks through the desired use case example. This should be similar to the labs we've done in class: you might want to demonstrate how to isolate a particularly interesting time period, then create an image showing a feature you're interested in, for example.

3) A discussion of the results and how they might be extended on further analysis. For example, we are doing El Nino/La Nina composites in class; a natural extension might be to look at individual events to see what their particular impacts were. Or if there are data quality issues which impact the results, you could discuss how these might be mitigated with additional information/analysis.

Just keep in mind, you'll have roughly 20 minutes for your full presentation, and that goes surprisingly quickly! Probably 2-3 diagnostics is the most you'll be able to get through (you could try practicing with your group members to get a sense of timing).


In [13]:
#select the region: Cacheu National Park Mangrove Forest

gb_lat = 12.1582165861
gb_lon = -16.283462202
pt = ee.Geometry.Point(gb_lon, gb_lat)

In [14]:
#filter image collection using our point in Chacheu National Park Mangrove Forest
gdat_filt = gdat.filterBounds(pt)

In [46]:
# Function to calculate NDVI for a given input image
def addNDVI(image):
    red = image.select('B4')
    nir = image.select('B5')
    
    ndvi = (nir.subtract(red)).divide((nir.add(red))).rename('NDVI')
    
    return image.addBands(ndvi)


# apply fuction to all
gdat_withndvi = gdat_filt.map(addNDVI) 

#create visable parameters 

ndviParams = {'bands': 'NDVI',
              'min': -1, 
              'max': 1, 
              'palette': ['blue', 'white', 'green']
             }

In [47]:
#filter out the cloudy images 
dat_nocld = gdat_withndvi.filter('CLOUD_COVER < 20')

#filter for the time period 2014 and create a temporal average 
date_2014 = dat_nocld.filter(ee.Filter.date('2014-01-01', '2014-12-31')).mean();

#creating the basemap 
map_ndvi = geemap.Map(center = [gb_lat, gb_lon], zoom = 12)

#add 2014 NDVI layer to map
map_ndvi.addLayer(date_2014, ndviParams, "2014")

In [48]:
#Now adding a 2019 layer 

#filter for the time period 2019 and create a temporal average 
date_2019 = dat_nocld.filter(ee.Filter.date('2019-01-01', '2019-12-31')).mean();

#add 2019 NDVI layer to map
map_ndvi.addLayer(date_2019, ndviParams, "2019")

In [49]:
#calling the map 
#map_ndvi

Map(center=[12.1582165861, -16.283462202], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [ ]:
#Now doing NDII 

def addNDII(image):
    mir = image.select('B6')
    nir = image.select('B5')
    
    ndii = (nir.subtract(mir)).divide((nir.add(mir))).rename('NDII')
    
    return image.addBands(ndii)

# apply fuction to all
gdat_withndii = gdat_filt.map(addNDII) 

#create visable parameters 

ndiiParams = {'bands': 'NDII',
              'min': -1, 
              'max': 1, 
              'palette': ['blue', 'white', 'green']
             }

#filter out the cloudy images 
dat_nocld_ndii = gdat_withndii.filter('CLOUD_COVER < 20')

#filter for the time period 2014 and create a temporal average 
date_2014_ndii = dat_nocld_ndii.filter(ee.Filter.date('2014-01-01', '2014-12-31')).mean();

#creating the basemap 
map_ndii = geemap.Map(center = [gb_lat, gb_lon], zoom = 12)

#add 2014 NDII layer to map
map_ndii.addLayer(date_2014_ndii, ndiiParams, "2014")

#Now adding a 2019 layer 

#filter for the time period 2019 and create a temporal average 
date_2019_ndii = dat_nocld_ndii.filter(ee.Filter.date('2019-01-01', '2019-12-31')).mean();

#add 2019 NDII layer to map
map_ndii.addLayer(date_2019_ndii, ndiiParams, "2019")

In [ ]:
#calling the map 
#map_ndii

In [ ]:
# Now doing the NDBI layer 

def addNDBI(image):
    mir = image.select('B6')
    nir = image.select('B5')
    
    ndbi = (mir.subtract(nir)).divide((mir.add(nir))).rename('NDBI')
    
    return image.addBands(ndbi)

# apply fuction to all
gdat_withndbi = gdat_filt.map(addNDBI) 

#create visable parameters 

ndbiParams = {'bands': 'NDBI',
              'min': -1, 
              'max': 1, 
              'palette': ['blue', 'white', 'green']
             }

#filter out the cloudy images 
dat_nocld_ndbi = gdat_withndbi.filter('CLOUD_COVER < 20')

#filter for the time period 2014 and create a temporal average 
date_2014_ndbi = dat_nocld_ndbi.filter(ee.Filter.date('2014-01-01', '2014-12-31')).mean();

#creating the basemap 
map_ndbi = geemap.Map(center = [gb_lat, gb_lon], zoom = 12)

#add 2014 NDBI layer to map
map_ndbi.addLayer(date_2014_ndbi, ndbiParams, "2014")

#Now adding a 2019 layer 

#filter for the time period 2019 and create a temporal average 
date_2019_ndbi = dat_nocld_ndbi.filter(ee.Filter.date('2019-01-01', '2019-12-31')).mean();

#add 2019 NDBI layer to map
map_ndbi.addLayer(date_2019_ndbi, ndbiParams, "2019")

In [ ]:
#calling the map 
#map_ndbi

In [18]:
# trying the GARI 

def addGARI(image):
    green = image.select('B3')
    blue = image.select('B2')
    red = image.select('B4')
    nir = image.select('B5')
    
    #gari = (nir.subtract(green.subtract((1.7).multiply(blue.subtratc(red)))).divide(nir.subtract(green.add(1.7.multiply(blue.subtract(red))))))
    
    return image.addBands(gari)

# apply fuction to all
gdat_withgari = gdat_filt.map(addGARI)

# the GARI isnt woring because it doesnt like the multiply 

AttributeError: 'float' object has no attribute 'multiply'

<a id='binder'></a> 
### Create Binder Environment

The last step is to create a Binder environment for your project, so that we don't have to spend time configuring everyone's environment each time we switch between group presentations. Instructions are below:

 - Assemble all of the data needed in your Github repo: Jupyter notebooks, a README file, and any datasets needed (these should be small, if included within the repo). Larger datasets should be stored on a separate server, and access codes included within the Jupyter notebook as discussed above. 
 
 - Create an _environment_ file: this is a text file which contains information on the packages needed in order to execute your code. The filename should be "environment.yml": an example that you can use for the proper syntax is included in this template repo. To determine which packages to include, you'll probably want to start by displaying the packages loaded in your environment: you can use the command `conda list -n [environment_name]` to get a list.
 
 More information on environment files can be found here:
 https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#

 - Create Binder. Use http://mybinder.org to create a  URL for your notebook Binder (you will need to enter your GitHub repo URL). You can also add a Launch Binder button directly to your GitHub repo, by including the following in your README.md:

```
launch with myBinder
[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/<path to your repo>)
```

<a id='references'></a> 
### References

List relevant references. Here are some additional resources on creating professional, shareable notebooks you may find useful:

1. Notebook sharing guidelines from reproducible-science-curriculum: https://reproducible-science-curriculum.github.io/publication-RR-Jupyter/
2. Guide for developing shareable notebooks by Kevin Coakley, SDSC: https://github.com/kevincoakley/sharing-jupyter-notebooks/raw/master/Jupyter-Notebooks-Sharing-Recommendations.pdf
3. Guide for sharing notebooks by Andrea Zonca, SDSC: https://zonca.dev/2020/09/how-to-share-jupyter-notebooks.html
4. Jupyter Notebook Best Practices: https://towardsdatascience.com/jupyter-notebook-best-practices-f430a6ba8c69
5. Introduction to Jupyter templates nbextension: https://towardsdatascience.com/stop-copy-pasting-notebooks-embrace-jupyter-templates-6bd7b6c00b94  
    5.1. Table of Contents (Toc2) readthedocs: https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/toc2/README.html  
    5.2. Steps to install toc2: https://stackoverflow.com/questions/23435723/installing-ipython-notebook-table-of-contents
6. Rule A, Birmingham A, Zuniga C, Altintas I, Huang SC, et al. (2019) Ten simple rules for writing and sharing computational analyses in Jupyter Notebooks. PLOS Computational Biology 15(7): e1007007. https://doi.org/10.1371/journal.pcbi.1007007. Supplementary materials: example notebooks (https://github.com/jupyter-guide/ten-rules-jupyter) and tutorial (https://github.com/ISMB-ECCB-2019-Tutorial-AM4/reproducible-computational-workflows)
7. Languages supported by Jupyter kernels: https://github.com/jupyter/jupyter/wiki/Jupyter-kernels
8. EarthCube notebooks presented at EC Annual Meeting 2020: https://www.earthcube.org/notebooks
9. Manage your Python Virtual Environment with Conda: https://towardsdatascience.com/manage-your-python-virtual-environment-with-conda-a0d2934d5195
10. Venv - Creation of Virtual Environments: https://docs.python.org/3/library/venv.html